In [ ]:
!pip3 install -r requirements.txt

## From the original code from Dongbin Na

#### <b>Download the Dataset</b>

* The dataset contains approximately 13,000 Korean "human face" images.
* In this dataset, all faces are cropped to a resolution of 128 X 128 pixels, although some of the original images have been high resolution.
* Each image filename indicates which family (household) number it belongs to.

In [2]:
%%capture
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/EbMhBPnmIb5MutZvGicPKggBWKm5hLs0iwKfGW7_TwQIKg?download=1 -O custom_korean_family_dataset_resolution_128.zip
!unzip custom_korean_family_dataset_resolution_128.zip -d ./custom_korean_family_dataset_resolution_128

#### <b>Download the Original Datasets</b>

* <b>CelebA-HQ Dataset</b>: A dataset consisting of 30,000 "human face" images.
    * This dataset has approximately 3GB of data in size and is one of the most widely used face datasets worldwide.
* <b>CelebA-HQ Identity Dataset</b>: A dataset that provides information about <b>who the individuals (subjects) are</b> for each facial image file.
* <b>CelebA-HQ Attribute Dataset</b>: A dataset that provides attribute information for each facial image file.
    * This attribute information includes <b>gender</b>, <b>smiling</b>, and <b>age</b>, which we will be used for a multi-label task.

In [3]:
!wget https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/Eb37jNPPA7hHl0fmktYqcV8B-qmPLx-ZKYQ1eFk4UPBV_A?download=1 -O CelebAMask-HQ.zip
!wget https://postechackr-my.sharepoint.com/:t:/g/personal/dongbinna_postech_ac_kr/EVRoUY8_txRFv56-KWvZrksBDWbD6adkjBxwwRN7qAC6bg?download=1 -O CelebA-HQ-identity.txt
!wget https://postechackr-my.sharepoint.com/:t:/g/personal/dongbinna_postech_ac_kr/EVrdIrPOkR1OlEWBVK8lE3AB9bFh741GnKBkNgPa8trNuA?download=1 -O CelebA-HQ-attribute.txt

--2024-01-17 10:54:18--  https://postechackr-my.sharepoint.com/:u:/g/personal/dongbinna_postech_ac_kr/Eb37jNPPA7hHl0fmktYqcV8B-qmPLx-ZKYQ1eFk4UPBV_A?download=1
Resolving postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to postechackr-my.sharepoint.com (postechackr-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/dongbinna_postech_ac_kr/Documents/Research/datasets/CelebAMask-HQ/CelebAMask-HQ.zip?ga=1 [following]
--2024-01-17 10:54:18--  https://postechackr-my.sharepoint.com/personal/dongbinna_postech_ac_kr/Documents/Research/datasets/CelebAMask-HQ/CelebAMask-HQ.zip?ga=1
Reusing existing connection to postechackr-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3153930546 (2.9G) [application/x-zip-compressed]
Saving to: ‘CelebAMask-HQ.zip’

CelebAMask-HQ.zip   100%[===================>]   2.94G   123MB/s    in

In [4]:
%%capture
!rm -rf ./CelebAMask-HQ
!unzip CelebAMask-HQ.zip

# 1. Creating a pre-trained model

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC.py \
    --num_epochs 30 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'teacher' \
    --optimizer 'adamw' \
    --manual_decay True

# 2. Finetuning with baseline

In [ ]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC.py \
    --num_epochs 30 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'ViT-B-16' \
    --transform 2 \
    --training_type 'finetune' \
    --optimizer 'adamw' \
    --manual_decay True \
    --save_teacher False \
    --checkpoint './MUFAC_ViT-B-16_teacher_seed_1.pth'

# 3. Finetuning with DASH

In [5]:
!CUDA_VISIBLE_DEVICES='0' python ViTMUL_MUFAC_ER.py \
    --num_epochs 20 \
    --seeds '1' \
    --learning_rate 0.00001 \
    --momentum 0.9 \
    --model_type 'ViT-B-16' \
    --transform 2 \
    --training_type 'finetune' \
    --optimizer 'adamw' \
    --manual_decay True \
    --save_teacher False \
    --checkpoint './MUFAC_ViT-B-16_teacher_seed_1.pth' \
    --DASH_layer 6

Train dataset size: 10025
Test dataset size: 1539
Forget dataset size: 1500
Retain dataset size: 8525
Unseen dataset size: 1504
seed 1
results saved at:  ./results/MUFAC_retrain_seed_1_output_0.tsv
-----------------------------------------------------------------------------
Using model type: retrain
-----------------------------------------------------------------------------
Using optimizer: adamw
-----------------------------------------------------------------------------
Using manual decay
-----------------------------------------------------------------------------
retrain (Seed: 1, Epoch: 0): 100%|████████████| 134/134 [00:37<00:00,  3.59it/s]
Epoch [1/30] - Total Loss: 16387.8337
[Test]
test loss: 0.029756943462265244, accuracy: 0.17543859649122806, top-2 accuracy: 0.4191033138401559
elapsed time: 2.2117183208465576
[Test]
test loss: 0.028757486254610914, accuracy: 0.18218085106382978, top-2 accuracy: 0.4454787234042553
elapsed time: 2.1466047763824463
[Test]
test loss: 0.03271

retrain (Seed: 1, Epoch: 6): 100%|████████████| 134/134 [00:35<00:00,  3.79it/s]
Epoch [7/30] - Total Loss: 6576.0860
[Test]
test loss: 0.016598961700317997, accuracy: 0.5886939571150097, top-2 accuracy: 0.8609486679662118
elapsed time: 2.2070083618164062
[Test]
test loss: 0.013366306081731269, accuracy: 0.6602393617021277, top-2 accuracy: 0.8962765957446809
elapsed time: 2.1571192741394043
[Test]
test loss: 0.019505682309468586, accuracy: 0.5073333333333333, top-2 accuracy: 0.7773333333333333
elapsed time: 2.142889976501465
-----------------------------------------------------------------------------
Seed 1, Epoch: 6
Utility score:  58.87
Forget score:  8.34
Forget-set accuracy:  50.73
NoMUS score:  71.10
-----------------------------------------------------------------------------
Best NoMUS score so far:  71.10 / Util:  58.87 / Forget:  8.34 / Forget Acc:  50.73
Exists tie: False
-----------------------------------------------------------------------------
Best test acc so far:  58.

retrain (Seed: 1, Epoch: 13): 100%|███████████| 134/134 [00:35<00:00,  3.80it/s]
Epoch [14/30] - Total Loss: 1891.9691
[Test]
test loss: 0.018626847752818355, accuracy: 0.6166341780376868, top-2 accuracy: 0.8875893437296946
elapsed time: 2.1839303970336914
[Test]
test loss: 0.014525009755124437, accuracy: 0.682845744680851, top-2 accuracy: 0.9228723404255319
elapsed time: 2.1418890953063965
[Test]
test loss: 0.022441364248593648, accuracy: 0.538, top-2 accuracy: 0.808
elapsed time: 2.140143871307373
-----------------------------------------------------------------------------
Seed 1, Epoch: 13
Utility score:  61.66
Forget score:  8.37
Forget-set accuracy:  53.80
NoMUS score:  72.46
-----------------------------------------------------------------------------
Best NoMUS score so far:  75.37 / Util:  60.04 / Forget:  4.65 / Forget Acc:  55.27
Exists tie: False
-----------------------------------------------------------------------------
Best test acc so far:  62.25
----------------------

retrain (Seed: 1, Epoch: 20): 100%|███████████| 134/134 [00:35<00:00,  3.79it/s]
Epoch [21/30] - Total Loss: 1048.0756
[Test]
test loss: 0.021256235104400358, accuracy: 0.6120857699805068, top-2 accuracy: 0.8778427550357375
elapsed time: 2.180920362472534
[Test]
test loss: 0.016504508938561094, accuracy: 0.6841755319148937, top-2 accuracy: 0.9089095744680851
elapsed time: 2.1116905212402344
[Test]
test loss: 0.0247261221408844, accuracy: 0.5486666666666666, top-2 accuracy: 0.804
elapsed time: 2.131969690322876
-----------------------------------------------------------------------------
Seed 1, Epoch: 20
Utility score:  61.21
Forget score:  5.71
Forget-set accuracy:  54.87
NoMUS score:  74.89
-----------------------------------------------------------------------------
Best NoMUS score so far:  75.37 / Util:  60.04 / Forget:  4.65 / Forget Acc:  55.27
Exists tie: False
-----------------------------------------------------------------------------
Best test acc so far:  62.38
-----------

retrain (Seed: 1, Epoch: 27): 100%|███████████| 134/134 [00:35<00:00,  3.78it/s]
Epoch [28/30] - Total Loss: 930.4695
[Test]
test loss: 0.021614741473479887, accuracy: 0.612735542560104, top-2 accuracy: 0.8784925276153346
elapsed time: 2.1912295818328857
[Test]
test loss: 0.01671325458966671, accuracy: 0.6855053191489362, top-2 accuracy: 0.910904255319149
elapsed time: 2.1299262046813965
[Test]
test loss: 0.02478707746664683, accuracy: 0.5506666666666666, top-2 accuracy: 0.8026666666666666
elapsed time: 2.1370363235473633
-----------------------------------------------------------------------------
Seed 1, Epoch: 27
Utility score:  61.27
Forget score:  7.08
Forget-set accuracy:  55.07
NoMUS score:  73.56
-----------------------------------------------------------------------------
Best NoMUS score so far:  75.37 / Util:  60.04 / Forget:  4.65 / Forget Acc:  55.27
Exists tie: False
-----------------------------------------------------------------------------
Best test acc so far:  62.38